# Modelado

In [ ]:
import joblib
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

# 1. Cargar los datos procesados
# Nota: No necesitamos el DataFrame original aquí, solo los números para la red.
X_train_scaled = joblib.load('data/processed/X_train_scaled.pkl')
X_test_scaled = joblib.load('data/processed/X_test_scaled.pkl')
y_train = joblib.load('data/processed/y_train.pkl')
y_test = joblib.load('data/processed/y_test.pkl')

print(f"Datos cargados correctamente.")
print(f"Input Shape (Features): {X_train_scaled.shape}")

In [ ]:
# 2. Definir la Arquitectura del Modelo (MLP)
model = Sequential()

# Capa de Entrada + Oculta 1 (64 neuronas)
# input_dim: Se ajusta automáticamente al número de columnas de X_train
model.add(Dense(64, activation='relu', input_dim=X_train_scaled.shape[1]))
model.add(Dropout(0.3)) # Apagamos el 30% para evitar memorización (Overfitting)

# Capa Oculta 2 (32 neuronas)
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))

# Capa Oculta 3 (16 neuronas - Refinamiento)
model.add(Dense(16, activation='relu'))

# Capa de Salida (1 neurona)
# Activation 'linear' porque queremos predecir un valor numérico continuo (Regresión)
model.add(Dense(1, activation='linear'))

# 3. Compilación
# Optimizer 'adam': El mejor estándar general.
# Loss 'mse': Mean Squared Error (Castiga errores grandes).
# Metrics 'mae': Mean Absolute Error (Más fácil de entender para humanos).
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

model.summary()

In [ ]:
# Definir el "Freno de Mano"
early_stop = EarlyStopping(
    monitor='val_loss',       # Vigila el error en datos de validación
    patience=10,              # Si no mejora en 10 épocas...
    restore_best_weights=True # ...vuelve al mejor momento y detente.
)

# Entrenar
print("Iniciando entrenamiento...")
history = model.fit(
    X_train_scaled, y_train,
    validation_split=0.2,     # Usa un 20% del train para validarse a sí misma mientras aprende
    epochs=100,               # Intenta hasta 100 veces
    batch_size=32,            # Procesa de a 32 clientes a la vez
    callbacks=[early_stop],
    verbose=1
)

print("Entrenamiento finalizado.")

In [ ]:
# Graficar la curva de pérdida (Loss)
plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Training Loss (MSE)')
plt.plot(history.history['val_loss'], label='Validation Loss (MSE)')
plt.title('Curva de Aprendizaje del Modelo CLTV')
plt.xlabel('Épocas')
plt.ylabel('Pérdida (MSE - Escala Log)')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
import os

# Crear carpeta de modelos
os.makedirs('models', exist_ok=True)

# Guardar el modelo entrenado
model.save('models/cltv_model_v1.keras')

print("Modelo guardado exitosamente en 'models/cltv_model_v1.keras'")
print("Listo para pasar al notebook de Evaluación.")